# Qwen 1.5B GRPO Training on Colab

Train **Qwen2.5-Coder-1.5B-Instruct** to fix Trapping Rain Water (80% → 100%).

## Runtime Selection Guide

| Runtime | VRAM | Approach | Speed |
|---------|------|----------|-------|
| **A100** | 40GB | Full GRPO | Fast |
| **L4** | 24GB | Full GRPO | Fast |
| **T4** | 16GB | LoRA+REINFORCE | Medium |
| **TPU** | 16GB | LoRA+REINFORCE | Slow |

**Recommended**: A100 or L4 for best results.

---

## Step 1: Check Runtime Type

Make sure you selected **TPU** runtime:
- Runtime -> Change runtime type -> TPU

In [ ]:
import os
import sys

print("=" * 60)
print("STEP 1: INSTALL DEPENDENCIES")
print("=" * 60)

# Install all dependencies
os.system("pip install -q transformers accelerate sentencepiece peft bitsandbytes")
print("Dependencies installed!")

## Step 2: Install Dependencies

In [ ]:
print("=" * 60)
print("STEP 2: DETECT RUNTIME & SELECT APPROACH")
print("=" * 60)

import torch

USE_FULL_GRPO = False  # Will be set based on GPU memory

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"✅ GPU: {gpu_name}")
    print(f"   Memory: {gpu_mem:.1f} GB")
    
    # Decide approach based on memory
    if gpu_mem >= 20:  # A100 (40GB) or L4 (24GB)
        USE_FULL_GRPO = True
        print(f"\n🚀 Using FULL GRPO (enough memory for 2x model copies)")
    else:  # T4 (16GB) or smaller
        USE_FULL_GRPO = False
        print(f"\n⚡ Using LoRA + REINFORCE (memory-efficient)")
else:
    device = torch.device("cpu")
    print("⚠️  No GPU found! Using CPU (very slow)")
    USE_FULL_GRPO = False

print(f"\nDevice: {device}")
print(f"Full GRPO: {USE_FULL_GRPO}")

## Step 3: Setup TPU Device

In [ ]:
import gc

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("Memory utilities defined!")

## Step 4: Configuration

In [ ]:
# Configuration based on runtime
if USE_FULL_GRPO:
    CONFIG = {
        "model_name": "Qwen/Qwen2.5-Coder-1.5B-Instruct",
        "num_steps": 10,
        "num_generations": 4,
        "learning_rate": 5e-5,
        "beta": 0.04,
        "max_new_tokens": 512,
        "difficulty": 5,
        "num_test_cases": 5,
        "seed": 42,
    }
else:
    CONFIG = {
        "model_name": "Qwen/Qwen2.5-Coder-1.5B-Instruct",
        "num_steps": 10,
        "num_generations": 2,      # Reduced for memory
        "learning_rate": 1e-4,     # Higher LR for LoRA
        "beta": 0.04,
        "max_new_tokens": 256,     # Reduced for memory
        "difficulty": 5,
        "num_test_cases": 5,
        "seed": 42,
        # LoRA config
        "lora_r": 16,
        "lora_alpha": 32,
        "lora_dropout": 0.05,
    }

print("=" * 60)
print("CONFIGURATION")
print("=" * 60)
for k, v in CONFIG.items():
    print(f"  {k:20} = {v}")

## Step 5: Problem Generator

In [6]:
import random
from dataclasses import dataclass
from typing import List, Any

@dataclass
class TestCase:
    input_args: List[Any]
    expected_output: Any

@dataclass 
class Problem:
    problem_id: str
    title: str
    description: str
    function_signature: str
    function_name: str
    test_cases: List[TestCase]
    
    def to_prompt(self) -> str:
        examples = "\n".join(
            f"  {self.function_name}({repr(tc.input_args[0])}) -> {repr(tc.expected_output)}"
            for tc in self.test_cases[:3]
        )
        return f"""## {self.title}

{self.description}

### Function Signature
```python
{self.function_signature}
```

### Examples
```python
{examples}
```

Write ONLY the function implementation."""


class TrappingRainWaterGenerator:
    def __init__(self, seed=None):
        self.rng = random.Random(seed)
        self._counter = 0
    
    def _solve(self, height):
        if not height: return 0
        left, right = 0, len(height) - 1
        left_max = right_max = water = 0
        while left < right:
            if height[left] < height[right]:
                if height[left] >= left_max: left_max = height[left]
                else: water += left_max - height[left]
                left += 1
            else:
                if height[right] >= right_max: right_max = height[right]
                else: water += right_max - height[right]
                right -= 1
        return water
    
    def generate(self, difficulty=5, num_test_cases=5):
        self._counter += 1
        test_cases = []
        for _ in range(num_test_cases):
            length = 5 + difficulty * 2
            heights = [self.rng.randint(0, 3 + difficulty) for _ in range(length)]
            test_cases.append(TestCase([heights], self._solve(heights)))
        
        return Problem(
            problem_id=f"trap_{self._counter}",
            title="Trapping Rain Water",
            description="Given n non-negative integers representing an elevation map, compute how much water it can trap after raining.",
            function_signature="def trap(height: list) -> int:",
            function_name="trap",
            test_cases=test_cases,
        )

# Test
gen = TrappingRainWaterGenerator(seed=42)
p = gen.generate()
print(f"Generated problem with {len(p.test_cases)} test cases")

Generated problem with 5 test cases


## Step 6: Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("=" * 60)
print("LOADING MODEL")
print("=" * 60)
print(f"Model: {CONFIG['model_name']}")
print(f"Mode: {'Full GRPO' if USE_FULL_GRPO else 'LoRA + REINFORCE'}")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded!")

if USE_FULL_GRPO:
    # Full precision model for A100/L4
    model = AutoModelForCausalLM.from_pretrained(
        CONFIG["model_name"],
        torch_dtype=torch.float16,
        device_map="auto",
    )
    print(f"Full model loaded (float16)")
else:
    # QLoRA for T4/smaller
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
    from transformers import BitsAndBytesConfig
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        CONFIG["model_name"],
        quantization_config=bnb_config,
        device_map="auto",
    )
    model = prepare_model_for_kbit_training(model)
    
    lora_config = LoraConfig(
        r=CONFIG["lora_r"],
        lora_alpha=CONFIG["lora_alpha"],
        lora_dropout=CONFIG["lora_dropout"],
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, lora_config)
    print(f"QLoRA model loaded (4-bit + LoRA adapters)")

# Stats
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"\nTrainable: {trainable/1e6:.1f}M / {total/1e9:.2f}B ({100*trainable/total:.2f}%)")

if torch.cuda.is_available():
    mem = torch.cuda.memory_allocated() / 1e9
    print(f"GPU Memory used: {mem:.2f} GB")

## Step 7: Helper Functions

In [ ]:
import re

def extract_code(response):
    for pattern in [r"```python\n(.*?)```", r"```\n(.*?)```"]:
        matches = re.findall(pattern, response, re.DOTALL)
        if matches: return matches[-1].strip()
    if "def " in response:
        lines, code, in_func = response.split("\n"), [], False
        for line in lines:
            if line.strip().startswith("def "): in_func = True
            if in_func: code.append(line)
        if code: return "\n".join(code).strip()
    return response

def verify(code, problem):
    namespace = {}
    try:
        exec(code, namespace)
    except: return False, 0.0
    
    func = namespace.get(problem.function_name)
    if not func:
        funcs = [v for k, v in namespace.items() if callable(v) and not k.startswith("_")]
        func = funcs[0] if funcs else None
    if not func: return False, 0.0
    
    passed = sum(1 for tc in problem.test_cases 
                 if _safe_call(func, tc.input_args) == tc.expected_output)
    return passed == len(problem.test_cases), passed / len(problem.test_cases)

def _safe_call(func, args):
    try: return func(*args)
    except: return None

def generate_solution(model, tokenizer, problem, temp=0.2):
    messages = [
        {"role": "system", "content": "You are an expert Python programmer."},
        {"role": "user", "content": problem.to_prompt()},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    model.eval()
    with torch.no_grad():
        out = model.generate(
            **inputs, 
            max_new_tokens=CONFIG["max_new_tokens"],
            do_sample=True, 
            temperature=temp, 
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("Helper functions defined!")

## Step 8: Baseline Evaluation

In [9]:
print("=" * 60)
print("BASELINE EVALUATION")
print("=" * 60)

gen = TrappingRainWaterGenerator(seed=CONFIG["seed"])
baseline_passed = 0

for i in range(5):
    p = gen.generate(CONFIG["difficulty"], CONFIG["num_test_cases"])
    response = generate_solution(model, tokenizer, p)
    code = extract_code(response)
    success, partial = verify(code, p)
    print(f"  [{i+1}] {'PASS' if success else 'FAIL'} ({partial*100:.0f}%)")
    baseline_passed += success
    clear_memory()

baseline_acc = baseline_passed / 5
print(f"\nBaseline: {baseline_passed}/5 ({baseline_acc*100:.0f}%)")

BASELINE EVALUATION


KeyboardInterrupt: 

## Step 9: Setup Optimizer (No Reference Model Needed!)

With LoRA, we skip the reference model copy. The base model weights are frozen, so we use simple REINFORCE.

In [ ]:
import copy

if USE_FULL_GRPO:
    print("=" * 60)
    print("CREATING REFERENCE MODEL (Full GRPO)")
    print("=" * 60)
    
    # Create frozen reference model copy
    ref_model = copy.deepcopy(model)
    ref_model.eval()
    for param in ref_model.parameters():
        param.requires_grad = False
    print("Reference model created!")
    
    # Optimizer for all parameters
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])
else:
    print("=" * 60)
    print("SETUP OPTIMIZER (LoRA - No Reference Model)")
    print("=" * 60)
    
    ref_model = None  # Not needed for REINFORCE
    print("Skipping reference model (using REINFORCE)")
    
    # Optimizer for LoRA params only
    optimizer = torch.optim.AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=CONFIG["learning_rate"]
    )

print(f"Optimizer: AdamW (lr={CONFIG['learning_rate']})")

if torch.cuda.is_available():
    mem = torch.cuda.memory_allocated() / 1e9
    print(f"GPU Memory: {mem:.2f} GB")

## Step 10: GRPO Training

In [ ]:
def grpo_step(model, ref_model, optimizer, problem):
    """Full GRPO step with reference model (for A100/L4)."""
    messages = [
        {"role": "system", "content": "You are an expert Python programmer."},
        {"role": "user", "content": problem.to_prompt()},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    prompt_len = inputs.input_ids.shape[1]
    
    # Generate completions
    completions, completion_ids = [], []
    model.eval()
    with torch.no_grad():
        for _ in range(CONFIG["num_generations"]):
            out = model.generate(
                **inputs, max_new_tokens=CONFIG["max_new_tokens"],
                do_sample=True, temperature=0.7, top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
            )
            completions.append(tokenizer.decode(out[0][prompt_len:], skip_special_tokens=True))
            completion_ids.append(out[0].detach())
    
    # Compute rewards
    rewards = []
    for comp in completions:
        code = extract_code(comp)
        success, partial = verify(code, problem)
        rewards.append(1.0 if success else partial * 0.5)
    
    rewards = torch.tensor(rewards, device=model.device)
    advantages = (rewards - rewards.mean()) / (rewards.std() + 1e-8)
    
    # Training
    model.train()
    optimizer.zero_grad()
    total_loss = 0.0
    
    for comp_ids, adv in zip(completion_ids, advantages):
        full_ids = comp_ids.unsqueeze(0)
        
        # Policy log probs
        outputs = model(input_ids=full_ids)
        logits = outputs.logits[:, prompt_len-1:-1, :]
        target = full_ids[:, prompt_len:]
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        seq_log_prob = torch.gather(log_probs, -1, target.unsqueeze(-1)).squeeze(-1).mean()
        
        # Reference log probs
        with torch.no_grad():
            ref_out = ref_model(input_ids=full_ids)
            ref_logits = ref_out.logits[:, prompt_len-1:-1, :]
            ref_log_probs = torch.nn.functional.log_softmax(ref_logits, dim=-1)
            ref_seq_log_prob = torch.gather(ref_log_probs, -1, target.unsqueeze(-1)).squeeze(-1).mean()
        
        # GRPO loss with KL penalty
        kl = seq_log_prob - ref_seq_log_prob
        loss = -adv * seq_log_prob + CONFIG["beta"] * kl
        loss.backward()
        total_loss += loss.item()
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    
    return {"loss": total_loss / CONFIG["num_generations"], 
            "mean_reward": rewards.mean().item(), "max_reward": rewards.max().item()}


def reinforce_step(model, optimizer, problem):
    """REINFORCE step without reference model (for T4/smaller)."""
    messages = [
        {"role": "system", "content": "You are an expert Python programmer."},
        {"role": "user", "content": problem.to_prompt()},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    prompt_len = inputs.input_ids.shape[1]
    
    # Generate completions
    completions, completion_ids = [], []
    model.eval()
    with torch.no_grad():
        for _ in range(CONFIG["num_generations"]):
            out = model.generate(
                **inputs, max_new_tokens=CONFIG["max_new_tokens"],
                do_sample=True, temperature=0.7, top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
            )
            completions.append(tokenizer.decode(out[0][prompt_len:], skip_special_tokens=True))
            completion_ids.append(out[0].detach())
    
    # Compute rewards
    rewards = []
    for comp in completions:
        code = extract_code(comp)
        success, partial = verify(code, problem)
        rewards.append(1.0 if success else partial * 0.5)
    
    rewards = torch.tensor(rewards, device=model.device)
    advantages = rewards - rewards.mean()
    if rewards.std() > 0:
        advantages = advantages / (rewards.std() + 1e-8)
    
    # Training
    model.train()
    optimizer.zero_grad()
    total_loss = 0.0
    
    for comp_ids, adv in zip(completion_ids, advantages):
        if adv.abs() < 1e-6: continue
        
        full_ids = comp_ids.unsqueeze(0)
        outputs = model(input_ids=full_ids)
        logits = outputs.logits[:, prompt_len-1:-1, :]
        target = full_ids[:, prompt_len:]
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        seq_log_prob = torch.gather(log_probs, -1, target.unsqueeze(-1)).squeeze(-1).mean()
        
        loss = -adv * seq_log_prob
        loss.backward()
        total_loss += loss.item()
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    
    return {"loss": total_loss / max(1, CONFIG["num_generations"]),
            "mean_reward": rewards.mean().item(), "max_reward": rewards.max().item()}


print("Training functions defined!")

In [ ]:
# Training loop
print("=" * 60)
print(f"TRAINING ({'Full GRPO' if USE_FULL_GRPO else 'REINFORCE'})")
print("=" * 60)
print(f"Steps: {CONFIG['num_steps']}, Generations: {CONFIG['num_generations']}")
print()

train_gen = TrappingRainWaterGenerator(seed=CONFIG["seed"] + 100)
metrics = []

for step in range(CONFIG["num_steps"]):
    problem = train_gen.generate(CONFIG["difficulty"], CONFIG["num_test_cases"])
    
    try:
        if USE_FULL_GRPO:
            m = grpo_step(model, ref_model, optimizer, problem)
        else:
            m = reinforce_step(model, optimizer, problem)
        
        metrics.append(m)
        print(f"Step {step+1:2d}: Loss={m['loss']:.4f}, Avg={m['mean_reward']:.3f}, Max={m['max_reward']:.3f}")
    except Exception as e:
        print(f"Step {step+1}: Error - {e}")
        import traceback
        traceback.print_exc()
    
    clear_memory()

print("\nTraining complete!")

## Step 11: Post-Training Evaluation

In [ ]:
print("=" * 60)
print("POST-TRAINING EVALUATION")
print("=" * 60)

eval_gen = TrappingRainWaterGenerator(seed=CONFIG["seed"] + 999)
final_passed = 0
model.eval()

for i in range(5):
    p = eval_gen.generate(CONFIG["difficulty"], CONFIG["num_test_cases"])
    response = generate_solution(model, tokenizer, p, temp=0.2)
    code = extract_code(response)
    success, partial = verify(code, p)
    print(f"  [{i+1}] {'PASS' if success else 'FAIL'} ({partial*100:.0f}%)")
    final_passed += success
    clear_memory()

final_acc = final_passed / 5
print(f"\nFinal: {final_passed}/5 ({final_acc*100:.0f}%)")

## Step 12: Results

In [ ]:
print("=" * 60)
print("RESULTS")
print("=" * 60)

print(f"\nBaseline: {baseline_acc*100:.0f}%")
print(f"Final:    {final_acc*100:.0f}%")
print(f"Change:   {(final_acc - baseline_acc)*100:+.0f}%")

if metrics:
    print(f"\nTraining curve:")
    for i, m in enumerate(metrics):
        print(f"  Step {i+1}: reward={m['mean_reward']:.3f}, max={m['max_reward']:.3f}")

if final_acc > baseline_acc:
    print("\n" + "="*60)
    print("SUCCESS! Model improved!")
    print("="*60)

## Step 13: Save Model (Optional)

In [ ]:
# Uncomment to save to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# 
# # Move model to CPU for saving
# model_cpu = model.to('cpu')
# save_path = "/content/drive/MyDrive/axiom-rl/qwen-1.5b-grpo-tpu"
# model_cpu.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)
# print(f"Saved to {save_path}")

---

## Notes

**TPU-specific considerations:**
- Use `xm.mark_step()` to sync TPU operations
- Use `xm.optimizer_step()` for optimizer updates
- TPU prefers float32 over float16
- First compilation is slow, subsequent runs are faster

**If training is slow:**
- This is normal for the first few steps (JIT compilation)
- TPU shines on larger batch sizes
- Consider using GPU (T4/A100) for this small-scale experiment